In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:



# Install required libraries
!pip install -q transformers datasets scikit-learn

# Import Libraries
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report
from torch.utils.data import Dataset

#  Load Data
train_df = pd.read_csv("/content/drive/MyDrive/NLPproj/train_2kmZucJ.csv")
test_df = pd.read_csv("/content/drive/MyDrive/NLPproj/test_oJQbWVk.csv")
sample_submission = pd.read_csv("/content/drive/MyDrive/NLPproj/sample_submission_LnhVWA4.csv")

print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)
train_df.head()

# Label distribution
print(train_df['label'].value_counts())

#Tokenization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

class TweetDataset(Dataset):
    def __init__(self, texts, labels=None):
        self.encodings = tokenizer(texts.tolist(), truncation=True, padding=True, max_length=128)
        self.labels = labels.tolist() if labels is not None else None

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

#  Train/Validation Split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_df["tweet"], train_df["label"], test_size=0.1, stratify=train_df["label"], random_state=42
)

train_dataset = TweetDataset(train_texts, train_labels)
val_dataset = TweetDataset(val_texts, val_labels)

#  Load Model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

## ✅ Training Arguments (fast config)
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="no",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_steps=10,
    report_to=[]  # disables W&B and other loggers
)



# ✅ Evaluation Metric
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    f1 = f1_score(labels, preds, average="weighted")
    return {"f1": f1}

# ✅ Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

import os
os.environ["WANDB_DISABLED"] = "true"

# ✅ Train
trainer.train()

# ✅ Inference on Test Set
test_dataset = TweetDataset(test_df["tweet"])
preds_output = trainer.predict(test_dataset)
test_preds = np.argmax(preds_output.predictions, axis=1)

# ✅ Prepare Submission
submission = sample_submission.copy()
submission["label"] = test_preds
submission.to_csv("submission.csv", index=False)

print("✅ Submission saved to 'submission.csv'")

# # ✅ Install required libraries
# #!pip install -q transformers datasets scikit-learn

# # ✅ Import Libraries
# import pandas as pd
# import numpy as np
# import torch
# from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import f1_score, classification_report
# from torch.utils.data import Dataset

# # ✅ Load Data
# train_df = pd.read_csv("/mnt/data/train_2kmZucJ.csv")
# test_df = pd.read_csv("/mnt/data/test_oJQbWVk.csv")
# sample_submission = pd.read_csv("/mnt/data/sample_submission_LnhVWA4.csv")

# print("Train shape:", train_df.shape)
# print("Test shape:", test_df.shape)
# train_df.head()

# # ✅ Label distribution
# print(train_df['label'].value_counts())

# # ✅ Encode labels if not already (assumes labels are 0 or 1)
# # If your labels are 'negative'/'not_negative', uncomment below:
# # train_df['label'] = train_df['label'].map({'not_negative': 0, 'negative': 1})

# # ✅ Tokenization
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# class TweetDataset(Dataset):
#     def __init__(self, texts, labels=None):
#         self.encodings = tokenizer(texts.tolist(), truncation=True, padding=True, max_length=128)
#         self.labels = labels.tolist() if labels is not None else None

#     def __getitem__(self, idx):
#         item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
#         if self.labels is not None:
#             item['labels'] = torch.tensor(self.labels[idx])
#         return item

#     def __len__(self):
#         return len(self.encodings["input_ids"])

# # ✅ Train/Validation Split
# train_texts, val_texts, train_labels, val_labels = train_test_split(
#     train_df["tweet"], train_df["label"], test_size=0.1, stratify=train_df["label"], random_state=42
# )

# train_dataset = TweetDataset(train_texts, train_labels)
# val_dataset = TweetDataset(val_texts, val_labels)

# # ✅ Load Model
# model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# # ✅ Training Arguments
# training_args = TrainingArguments(
#     output_dir="./results",
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     logging_dir="./logs",
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     num_train_epochs=3,
#     weight_decay=0.01,
#     load_best_model_at_end=True,
#     metric_for_best_model="f1",
# )

# # ✅ Evaluation Metric
# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     preds = np.argmax(logits, axis=1)
#     f1 = f1_score(labels, preds, average="weighted")
#     return {"f1": f1}

# # ✅ Trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
#     compute_metrics=compute_metrics
# )

# # ✅ Train
# trainer.train()

# # ✅ Inference on Test Set
# test_dataset = TweetDataset(test_df["tweet"])
# preds_output = trainer.predict(test_dataset)
# test_preds = np.argmax(preds_output.predictions, axis=1)

# # ✅ Prepare Submission
# submission = sample_submission.copy()
# submission["label"] = test_preds
# submission.to_csv("submission.csv", index=False)

# print("✅ Submission saved to 'submission.csv'")

Train shape: (7920, 3)
Test shape: (1953, 2)
label
0    5894
1    2026
Name: count, dtype: int64


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,F1
1,0.220900,0.181679,0.924379


✅ Submission saved to 'submission.csv'
